In [1]:
# Use PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.conv2 = nn.Conv2d(16, 16, 3)
        self.fc1 = nn.Linear(256, 16)
        self.fc2 = nn.Linear(16, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 256)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)
        

In [3]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    total_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        total_loss.append(loss.item())
    print ('Epoch {}, average loss = {}'.format(epoch, avg(total_loss)))

In [4]:
def test(args, model, device, test_loader):
    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    correct = 1. * correct / len(test_loader.dataset)
    print ('Test loss = {}, accuracy = {}%'.format(test_loss, correct * 100.))

In [5]:
def main():
    device = torch.device('cpu')
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True)
    model = Network().to(device)
    optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.5)
    
    args = {} # Extra args
    for epoch in range(10):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

In [6]:
if __name__ == '__main__':
    main()

/Users/yftung/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


ValueError: Expected input batch_size (2304) to match target batch_size (64).